In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import cv2
import os

# 이미지 파일이 있는 폴더 경로
image_folder = "/home/addinedu/dev_ws/data/ox_images/train"
# 라벨 CSV 파일 경로
label_csv_path = "/home/addinedu/dev_ws/data/ox_images/train_labels.csv"

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (32, 32))  # 이미지 크기를 32x32로 조정
    image = image / 255.0  # 이미지를 0과 1 사이의 값으로 정규화
    return image

# 데이터셋 생성 함수
def create_dataset(image_folder, label_csv_path):
    # 라벨 CSV 파일 로드
    label_df = pd.read_csv(label_csv_path)
    
    images = []
    labels = []

    # 각 이미지에 대해 라벨을 매핑하여 이미지와 라벨 데이터 생성
    for index, row in label_df.iterrows():
        image_path = os.path.join(image_folder, row["Image_Path"])
        label = row["Label"]
        image = load_and_preprocess_image(image_path)
        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

# 데이터셋 생성
images, labels = create_dataset(image_folder, label_csv_path)

# 이미지 데이터와 라벨 데이터를 훈련 및 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# 클래스 개수 계산
num_classes = len(np.unique(labels))

# CNN 모델 생성
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),  # 배치 정규화 추가
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import cv2
import os

# 이미지 파일이 있는 폴더 경로
image_folder = "/home/addinedu/dev_ws/data/ox_images/train"
# 라벨 CSV 파일 경로
label_csv_path = "/home/addinedu/dev_ws/data/ox_images/train_labels.csv"

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (32, 32))  # 이미지 크기를 32x32로 조정
    image = image / 255.0  # 이미지를 0과 1 사이의 값으로 정규화
    return image

# 데이터셋 생성 함수
def create_dataset(image_folder, label_csv_path):
    # 라벨 CSV 파일 로드
    label_df = pd.read_csv(label_csv_path)
    
    images = []
    labels = []

    # 각 이미지에 대해 라벨을 매핑하여 이미지와 라벨 데이터 생성
    for index, row in label_df.iterrows():
        image_path = os.path.join(image_folder, row["File_Name"])
        label = row["Label"]
        if label == 'O':
            label = 0
        elif label == 'x':
            label = 1
        image = load_and_preprocess_image(image_path)
        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

# 데이터셋 생성
images, labels = create_dataset(image_folder, label_csv_path)

# 이미지 데이터와 라벨 데이터를 훈련 및 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(images, labels, shuffle=True,test_size=0.2, random_state=42)

# 클래스 개수 계산
num_classes = len(np.unique(labels))

# CNN 모델 생성
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),  # 배치 정규화 추가
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(num_classes, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


모델저장

In [ ]:
# 다른 디렉토리에 훈련된 모델 저장
model.save("/home/addinedu/dev_ws/DL/ox_model.h5")

예측

In [ ]:
import cv2
import numpy as np

# 단일 이미지 전처리 함수
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (32, 32))
    image = image / 255.0
    return image

# 단일 이미지에 대한 라벨 예측 함수
def predict_image(image_path, model):
    preprocessed_image = preprocess_image(image_path)
    # 모델 입력 모양에 맞게 차원 확장
    input_image = np.expand_dims(preprocessed_image, axis=0)
    # 각 클래스에 대한 확률 예측
    predictions = model.predict(input_image)
    # 가장 높은 확률을 가진 인덱스 가져오기
    predicted_class = np.argmax(predictions)
    return predicted_class

# 훈련된 모델 로드
model = tf.keras.models.load_model('/home/addinedu/dev_ws/DL/ox_model.h5')

# 예제 사용법
image_path = '/home/addinedu/dev_ws/data/ox_images/test/O_10.jpg'
predicted_class = predict_image(image_path, model)

# 클래스 인덱스와 클래스 라벨 간의 매핑 정의
class_mapping = {0: 'O', 1: 'X'}

# 예측된 라벨 가져오기
predicted_label = class_mapping[predicted_class]

print("이미지에 대한 예측된 라벨:", predicted_label)


평가

In [ ]:
# 테스트 이미지 폴더 경로
test_image_folder = "/home/addinedu/dev_ws/data/ox_images/test"
test_label_csv_path = "/home/addinedu/dev_ws/data/ox_images/test_labels.csv"

# 테스트 데이터셋 생성 함수
def create_test_dataset(image_folder):
    test_images = []

    # 테스트 이미지 로드 및 전처리
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"):  # jpg 파일만 처리
            image_path = os.path.join(image_folder, filename)
            image = load_and_preprocess_image(image_path)
            test_images.append(image)

    return np.array(test_images)

# 테스트 데이터셋 생성
test_images = create_test_dataset(test_image_folder)

# 모델을 사용하여 테스트 데이터에 대한 예측 수행
predictions = model.predict(test_images)

# 예측 결과 확인
for i, prediction in enumerate(predictions):
    # 예측 결과 중 가장 높은 확률을 가지는 클래스 선택
    predicted_label = np.argmax(prediction)
    print(f"Image {i+1}: Predicted Label: {predicted_label}")


# 예측 결과 확인 및 정확도 계산
correct_count = 0
total_count = len(test_labels)

for i, (prediction, true_label) in enumerate(zip(predictions, test_labels)):
    # 예측 결과 중 가장 높은 확률을 가지는 클래스 선택
    predicted_label = np.argmax(prediction)
    
    # 예측 결과와 정답 비교하여 정확도 계산
    if predicted_label == true_label:
        correct_count += 1
    
    print(f"Image {i+1}: Predicted Label: {predicted_label}, True Label: {true_label}")

# 정확도 출력
accuracy = correct_count / total_count
print(f"Accuracy: {accuracy}")



In [ ]:
import pandas as pd

# 라벨 CSV 파일 로드
label_df = pd.read_csv(label_csv_path)

# 열 이름 확인
print(label_df.columns)

# 실제 열 이름 확인 후 코드 수정
# 예를 들어, 'Image Path' 열을 사용한다고 가정하면
# image_path = os.path.join(image_folder, row["Image Path"])
# label = row["Label"]


In [15]:
import os
import pandas as pd

# 테스트 이미지 폴더 경로
test_image_folder = "/home/addinedu/dev_ws/data/ox_images/test"
test_label_csv_path = "/home/addinedu/dev_ws/data/ox_images/test_labels.csv"

# 테스트 데이터셋 생성 함수
def create_test_dataset(image_folder):
    test_images = []

    # 테스트 이미지 로드 및 전처리
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"):  # jpg 파일만 처리
            image_path = os.path.join(image_folder, filename)
            image = load_and_preprocess_image(image_path)
            test_images.append(image)

    return np.array(test_images)

# 테스트 데이터셋 생성
test_images = create_test_dataset(test_image_folder)

# 모델을 사용하여 테스트 데이터에 대한 예측 수행
predictions = model.predict(test_images)

# 예측 결과 확인
for i, prediction in enumerate(predictions):
    # 예측 결과 중 가장 높은 확률을 가지는 클래스 선택
    predicted_label = np.argmax(prediction)
    print(f"Image {i+1}: Predicted Label: {predicted_label}")

# 테스트 라벨 CSV 파일 생성
test_labels = []
file_names = []

for filename in os.listdir(test_image_folder):
    if filename.endswith(".jpg"):  # jpg 파일만 처리
        file_names.append(filename)
        label = filename[0]  # 파일명의 첫 글자를 라벨로 설정
        if label == 'O':
            test_labels.append(0)
        elif label == 'x':
            test_labels.append(1)

test_data = pd.DataFrame({"File_Name": file_names, "Label": test_labels})
test_data.to_csv(test_label_csv_path, index=False)

print("테스트 라벨이 CSV 파일로 저장되었습니다:", test_label_csv_path)

# 예측 결과 확인 및 정확도 계산
correct_count = 0
total_count = len(test_labels)

for i, (prediction, true_label) in enumerate(zip(predictions, test_labels)):
    # 예측 결과 중 가장 높은 확률을 가지는 클래스 선택
    predicted_label = np.argmax(prediction)
    
    # 예측 결과와 정답 비교하여 정확도 계산
    if predicted_label == true_label:
        correct_count += 1
    
    print(f"Image {i+1}: Predicted Label: {predicted_label}, True Label: {true_label}")

# 정확도 출력
accuracy = correct_count / total_count
print(f"Accuracy: {accuracy}")


NameError: name 'load_and_preprocess_image' is not defined

: 

실시간 웹캠 테스트

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# 텐서플로우 모델 로드
model = tf.keras.models.load_model('/home/addinedu/dev_ws/DL/ox_model.h5')  # 모델 파일 경로 지정

def preprocess_image(img):
    # 이미지를 32x32로 조정하고, 0에서 1 사이의 값을 갖도록 정규화
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def detect_number(img, model):
    # 이미지 전처리
    processed_img = preprocess_image(img)
    processed_img = np.expand_dims(processed_img, axis=-1)  # 이미지를 3차원으로 확장

    
    # 모델을 사용하여 예측 진행
    prediction = model.predict(processed_img)
    predicted_label = np.argmax(prediction)
    
    return predicted_label

cap = cv2.VideoCapture(0)

if cap.isOpened():
    while True:
        ret, frame = cap.read()
        if ret:
            # 이미지 전처리: 그레이 스케일 및 이진화
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray, 110, 255, cv2.THRESH_BINARY_INV)
            
            # 숫자 감지 및 출력
            predicted_label = detect_number(binary, model)
            cv2.putText(frame, str(predicted_label), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow('Object Detection', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("No Frame...")
            break
else:
    print("Camera not opened...")

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# 텐서플로우 모델 로드
model = tf.keras.models.load_model('/home/addinedu/dev_ws/DL/ox_model.h5')  # 모델 파일 경로 지정

def preprocess_image(img):
    # 이미지를 32x32로 조정하고, 0에서 1 사이의 값을 갖도록 정규화
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    img = img.astype("float32") / 255.0
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # 이미지를 그레이스케일에서 RGB로 변환
    return img


def detect_number(img, model):
    # 이미지 전처리
    processed_img = preprocess_image(img)
    processed_img = np.expand_dims(processed_img, axis=0)  # 이미지를 4차원으로 확장
    
    # 모델을 사용하여 예측 진행
    prediction = model.predict(processed_img)
    predicted_label = np.argmax(prediction)
    
    return predicted_label

cap = cv2.VideoCapture(0)

if cap.isOpened():
    while True:
        ret, frame = cap.read()
        if ret:
            # 이미지 전처리: 그레이 스케일 및 이진화
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray, 110, 255, cv2.THRESH_BINARY_INV)
            
            # 숫자 감지 및 출력
            predicted_label = detect_number(binary, model)
            cv2.putText(frame, str(predicted_label), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow('Object Detection', binary)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("No Frame...")
            break
else:
    print("Camera not opened...")

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# 텐서플로우 모델 로드
model = tf.keras.models.load_model('/home/addinedu/dev_ws/DL/ox_model.h5')  # 모델 파일 경로 지정

def preprocess_image(img):
    # 이미지를 32x32로 조정하고, 0에서 1 사이의 값을 갖도록 정규화
    if img.shape[0] < 32 or img.shape[1] < 32:
        raise ValueError("이미지 크기가 너무 작습니다. 이미지 크기를 늘려주세요.")
    
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    img = img.astype("float32") / 255.0
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # 이미지를 그레이스케일에서 RGB로 변환
    return img


def detect_number(img, model):
    # 이미지 전처리
    processed_img = preprocess_image(img)
    processed_img = np.expand_dims(processed_img, axis=0)  # 이미지를 4차원으로 확장
    
    # 모델을 사용하여 예측 진행
    prediction = model.predict(processed_img)
    predicted_label = np.argmax(prediction)
    
    return predicted_label

cap = cv2.VideoCapture(2)

def nothing(x):
    pass

#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

cv2.namedWindow("Trackbars")

cv2.createTrackbar("Min Radius", "Trackbars", 16, 30, nothing)
cv2.createTrackbar("Max Radius", "Trackbars", 22, 80, nothing)
cv2.createTrackbar("Distance Threshold", "Trackbars", 50, 200, nothing)


if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            min_radius = cv2.getTrackbarPos("Min Radius", "Trackbars")
            max_radius = cv2.getTrackbarPos("Max Radius", "Trackbars")
            distance_threshold = cv2.getTrackbarPos("Distance Threshold", "Trackbars")
            
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
            thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY)
            bin_img = cv2.bitwise_not(bin_img)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            digits = []
            
            for contour in contours:
                (x, y), radius = cv2.minEnclosingCircle(contour)
                if 10 < radius < 40:
                    xs, xe = int(x-radius), int(x+radius)
                    ys, ye = int(y-radius), int(y+radius)
                    cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                    roi = bin_img[ys:ye, xs:xe]
                    
                    # 숫자 감지 및 예측
                    predicted_label = detect_number(roi, model)
                    digits.append((predicted_label, x))
                    
                    cv2.putText(bin_img, str(predicted_label), (xs, ys), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow('Object Detection', bin_img)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("프레임 없음...")
            break
else:
    print("카메라가 열리지 않았습니다...")

cap.release()
cv2.destroyAllWindows()


실시간 웹캠은 정상 작동된다. 하지만 예측을 안함 ㅋ

In [2]:
import cv2
import numpy as np
import tensorflow as tf
model = tf.keras.models.load_model('/home/addinedu/dev_ws/DL/Lenet1_model.h5')  # 모델 파일 경로 지정

def nothing(x):
    pass
cap = cv2.VideoCapture(2)

#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

cv2.namedWindow("Trackbars")

cv2.createTrackbar("Min Radius", "Trackbars", 16, 30, nothing)
cv2.createTrackbar("Max Radius", "Trackbars", 22, 80, nothing)

def preprocess_image(img):
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    img = img.astype("float32") / 255.0
    return img



if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            min_radius = cv2.getTrackbarPos("Min Radius", "Trackbars")
            max_radius = cv2.getTrackbarPos("Max Radius", "Trackbars")
            
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
            # thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY)
            
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                                   cv2.CHAIN_APPROX_SIMPLE)
            digits = []
            #number_counts = {}
            try:
                for contour in contours:
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if 20 < radius < 200:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        
                        roi = cv2.resize(roi, (32, 32), interpolation=cv2.INTER_AREA)
                        roi = roi.astype("float32") / 255.0
                        roi = np.expand_dims(roi, axis=0)
                        
                        # PyTorch 텐서로 변환
                        processed_roi = preprocess_image(roi)
                        processed_roi = np.expand_dims(processed_roi, axis=0)
                    
                        # 모델을 사용하여 예측 진행
                        prediction = model.predict(processed_roi)
                        num = np.argmax(prediction)
                        
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
            
            except Exception as e:
                pass

            cv2.imshow("Video Capture", bin_img)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("No Frame...")
            break
else:
    print("Camera not opened...")
cap.release()
cv2.destroyAllWindows() 

캡쳐하여 전처리 후 예측

In [14]:
import cv2
import numpy as np
import tensorflow as tf

# 모델 로드
model = tf.keras.models.load_model('/home/addinedu/dev_ws/DL/cnn1_model.h5')

def preprocess_image(img):
    img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
    img = img.astype("float32") / 255.0
    return img

# 웹캠 캡처
cap = cv2.VideoCapture(2)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    cv2.imshow('Webcam', frame)
    
    key = cv2.waitKey(1)
    
    # 'c' 키를 누르면 캡쳐하고 예측
    if key == ord('c'):
        processed_img = preprocess_image(frame)
        # 모델을 사용하여 값을 예측
        prediction = model.predict(np.array([processed_img.reshape(32, 32, 1)]) / 255.0)
        # 예측 결과 출력
        print("Predicted Value:", np.argmax(prediction))
    
    # 'q' 키를 누르면 종료
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d_21" is incompatible with the layer: expected axis -1 of input shape to have value 3, but received input with shape (1, 32, 32, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 32, 32, 1), dtype=float32)
  • training=False
  • mask=None